<a href="https://colab.research.google.com/github/YolaYing/zk-toolkit/blob/main/Plonk%2BKZG_Implementation(Python_Version).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Develope Environment Preparation
Curve we used is BLS12-381
- Library we used: BLS21-381 curve implemented by Ethereum, using Python
- Lib link: https://github.com/ethereum/py_ecc/tree/main

You can use the following command to install all the packages we needed. Note that if you are using anaconda, package installation may be failed. Highly recommand using colab or some development friendly environment.

In [1]:
!pip3 install py_ecc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 859.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.4 MB/s eta 0:00:00


# Preliminaries


## Form of Polynomial Representations

A polynomial $\Phi(x) = \sum_{i = 0}^{n - 1} \phi_ix^i$ have two representation forms:
1. Coefficient Form
  - $\Phi(x)$ can be represeneted as a tuple of $n$ coefficients: $[\phi_0, \phi_1, ..., \phi_{n-1}]$
2. Evaluation Form
  - $\Phi(x)$ can be represeneted as a tuple of $n$ distinct evaluations: $[\Phi(x_0), \Phi(x_1), ..., \Phi(x_{n-1})]$
    - the set of values $\\{ x_0, x_1, ..., x_{n-1} \\}$ over which the polynomial is defined over is known as "evaluation domain"

In [14]:
# For example, assume we have a polynomial Φ(𝑥)=4x^3+5x^2+3x+2 and some points on the polynomials
poly_coefs = [2, 3, 5, 4]
evaluation_domain = [1, 4, 16, 13]
poly_evals = [14, 10, 0, 1]

print(f'coefficient form of phi(x) is {poly_coefs}')
print(f'evaluation form of phi(x) is {poly_evals}, defined over evaluation domain {evaluation_domain}')

coefficient form of phi(x) is [2, 3, 5, 4]
evaluation form of phi(x) is [14, 10, 0, 1], defined over evaluation domain [1, 4, 16, 13]


## Convert between Coefficient Form and Evaluation Form

### Defination
- **Fourier Transform**: convert from coefficient to evaluation
- **inverse Fourier Transform**: convert from evaluation to coefficient


### Naive Way
In naive way, each of those two transformation takes $O(n^2)$ computation
- **Fourier Transform**: evaluate $\Phi(x)$ at each $x_i$ in the evaluation domain
- **inverse Fourier Transform**: use *Lagrange Interpolation* to obtain unique degree $(n-1)$ polynomial passes through each of the $n$ points


### Optimized Way: Fast Fourier Transformation(FFT)
To improve the effectiveness of the transformation, we need to do the following steps:
- defined polynomials over finite field
  - restrict each coefficient $p_i \in F_q$ and each evaluation point $\Phi(x_i) \in F_q$
- defined the evaluation domain as a multiplicative subgroup of $F_q$
  - evaluation domain is a set of "$n^{th}$ roots of unity", $\{ \omega_0, \omega_1, ..., \omega_{n-1} \}$ for some element $\omega \in F_q$ with order $n$(i.e. $\omega^n = 1$ mod $q$)
- implementation of FFT algorithm
  - if you are not familiar with FFT, highly recommend this video: https://www.youtube.com/watch?v=h7apO7q16V0

In [15]:
import random

def FFT(poly_coefs, q):
  '''
  Args:
  poly_coefs: coefficient representation of the polynomial

  Returns:
  y: evaluation form of the polynomial
  '''
  # get the degree of the polynomial
  n = len(poly_coefs)
  if n == 1:
    return poly_coefs

  # in theory, omega should be the nth root of unity, which is a complex number
  # to use it in finite field, we use a algorithm here
  omega = find_nth_root_of_unity(n, q)
  print(f'{n}^th of unity is {omega}')
  poly_coefs_e = poly_coefs[::2]
  poly_coefs_o = poly_coefs[1::2]
  y_e = FFT(poly_coefs_e, q)
  y_o = FFT(poly_coefs_o, q)
  y = [0] * n
  for j in range(int(n/2)):
    y[j] = int(y_e[j] + (omega**j)*y_o[j]) % q
    y[j + int(n/2)] = int(y_e[j] - (omega**j)*y_o[j]) % q
  return y

def find_nth_root_of_unity(n, q):
  '''
  Args:
  n: nth root of unity, which is the degree of polynomial
  q: finite field q

  Returns:
  omega: the nth root of unity, which is a element in finite field
  '''
  omega = 1
  while(omega**(n/2) == 1 and omega != 0):
    x = random.randint(0, q)
    omega = x**((q-1)/n) % q
  return omega

print(f'coefficient form of phi(x) is {poly_coefs}, after FFT, we can get the evaluation form of phi(x) is {FFT(poly_coefs, 17)}')

4^th of unity is 13.0
2^th of unity is 16.0
2^th of unity is 0.0
coefficient form of phi(x) is [2, 3, 5, 4], after FFT, we can get the evaluation form of phi(x) is [14, 1, 0, 10]


In [17]:
# verification of FFT result
omega = 13
q = 17
for i in range(len(poly_coefs)):
    x = omega**i % q
    print(f'x = {x}, poly(x) = {(4*(x**3)+5*(x**2)+3*x+2) % q}')

x = 1, poly(x) = 14
x = 13, poly(x) = 1
x = 16, poly(x) = 0
x = 4, poly(x) = 10


Because the calculation of finding evaluation domain is a repeated and computation intensive step, so usually we just build a lookup table to store the pre-calculated result. Here we just slightly revise FFT algorithm to meet the lookup needs.

In [18]:
# omega generation can be quite computational intensive, so we tend to pre-calculate the lookup table for omega
# we assume n = 8, and we have a multiplication cyclic group with ord = 8, which is [1,2,4,8,16,15,13,9] with 2 as its generator and 17 as module

# create a lookup table
n_max = 8
q = 17
omega_list = [1,2,4,8,16,15,13,9]

def build_lookup_table(n, omega_list):
  lookup = {}
  lookup[n] = omega_list
  while n > 2:
    n = int(n/2)
    omega_list = omega_list[::2]
    lookup[n] = omega_list
  return lookup

lookup  = build_lookup_table(n_max, omega_list)
print(f'lookup table of list of omega is {lookup}')

lookup table of list of omega is {8: [1, 2, 4, 8, 16, 15, 13, 9], 4: [1, 4, 16, 13], 2: [1, 16]}


In [20]:
# revise FFT using omega lookup table
def FFT_using_exist_omega(poly_coefs, q, lookup):
  '''
  Args:
  poly_coefs: coefficient representation of the polynomial
  lookup: pre-determined omega list

  Returns:
  y: evaluation form of the polynomial
  '''
  # get the degree of the polynomial
  n = len(poly_coefs)
  if n == 1:
    return poly_coefs

  # in theory, omega should be the nth root of unity, which is a complex number
  # to use it in finite field, we use a algorithm here
  omega = lookup[n][1]
  # print(f'n = {n}, omega = {omega}')
  poly_coefs_e = poly_coefs[::2]
  poly_coefs_o = poly_coefs[1::2]
  y_e = FFT_using_exist_omega(poly_coefs_e, q, lookup)
  y_o = FFT_using_exist_omega(poly_coefs_o, q, lookup)
  y = [0] * n
  for j in range(int(n/2)):
    y[j] = int(y_e[j] + (omega**j)*y_o[j]) % q
    y[j + int(n/2)] = int(y_e[j] - (omega**j)*y_o[j]) % q
  return y

print(f'coefficient form of phi(x) is {poly_coefs}, after FFT(use exist omega lookup table), we can get the evaluation form of phi(x) is {FFT_using_exist_omega(poly_coefs, q, lookup)}')

coefficient form of phi(x) is [2, 3, 5, 4], after FFT(use exist omega lookup table), we can get the evaluation form of phi(x) is [14, 10, 0, 1]


### Inverse Fast Fourier Transformation(IFFT)
We have used FFT to achieve fast transformation from coefficient to evaluation. Now we will implement the inverse transformation, which is from evaluation to coefficient with slightly changing in the algorithm: update $\omega$ to $\frac{1}{n}\omega^{-1}$

detailed info: https://decentralizedthoughts.github.io/2023-09-01-FFT/#mjx-eqn-%5Cstar

In [37]:
# inverse the lookup table
# the only thing we need to modify in FFT is to update its omega
omega_list = [1,2,4,8,16,15,13,9]

def inverse_omega_list(omega_list, q):
  inverse_omega_list = []
  inverse_omega_dict = {}

  for i in omega_list:
    for candidate in omega_list:
      if i*candidate%q == 1:
        inverse_omega_list.append(candidate)
        inverse_omega_dict[i] = candidate
  return inverse_omega_list, inverse_omega_dict

inverse_omega_list, inverse_omega_dict = inverse_omega_list(omega_list, q)
print(f'inverse omega list is {inverse_omega_list}, inverse omega dictionary is {inverse_omega_dict}')
inverse_lookup = build_lookup_table(n_max, inverse_omega_list)
print(f'lookup table of list of omega is {inverse_lookup}')

inverse omega list is [1, 9, 13, 15, 16, 8, 4, 2], inverse omega dictionary is {1: 1, 2: 9, 4: 13, 8: 15, 16: 16, 15: 8, 13: 4, 9: 2}
lookup table of list of omega is {8: [1, 9, 13, 15, 16, 8, 4, 2], 4: [1, 13, 16, 4], 2: [1, 16]}


In [48]:
# revise FFT using omega lookup table
def IFFT_using_exist_omega(poly_evals, q, inverse_lookup, inverse_omega_dict):
  recursion_result = IFFT_recursion_part(poly_evals, q, inverse_lookup)
  IFFT_final_result = [ x * inverse_omega_dict[len(poly_evals)] % q for x in recursion_result]
  return IFFT_final_result

def IFFT_recursion_part(poly_evals, q, inverse_lookup):
  '''
  Args:
  poly_coefs: coefficient representation of the polynomial
  lookup: pre-determined omega list

  Returns:
  y: evaluation form of the polynomial
  '''
  # get the degree of the polynomial
  n = len(poly_evals)
  if n == 1:
    return poly_evals

  # in theory, omega should be the nth root of unity, which is a complex number
  # to use it in finite field, we use a algorithm here
  omega = inverse_lookup[n][1]
  poly_evals_e = poly_evals[::2]
  poly_evals_o = poly_evals[1::2]
  y_e = IFFT_recursion_part(poly_evals_e, q, inverse_lookup)
  y_o = IFFT_recursion_part(poly_evals_o, q, inverse_lookup)
  y = [0] * n
  for j in range(int(n/2)):
    y[j] = int(y_e[j] + (omega**j)*y_o[j]) % q
    y[j + int(n/2)] = int(y_e[j] - (omega**j)*y_o[j]) % q
  return y

print(f'evaluation form of phi(x) is {poly_evals}, after IFFT(use exist omega lookup table), we can get the evaluation form of phi(x) is {IFFT_using_exist_omega(poly_evals, q, inverse_lookup, inverse_omega_dict)}')

evaluation form of phi(x) is [14, 10, 0, 1], after IFFT(use exist omega lookup table), we can get the evaluation form of phi(x) is [2, 3, 5, 4]


# KZG Implementation(Python Version)

The KZG Commitment Scheme is a commitment scheme that allows to commit to a polynomial $\Phi(x) = \phi_0 +\phi_1x+\phi_2x^2+...+\phi_lx^l$, where $\Phi(x) \in F_p[x]$ . 'to commit' means proving that you know the polynomial $\Phi(x)$ without revealing it.

The KZG commitment scheme consists of 4 steps:
1. Setup
2. Commit to Polynomials
3. Prove an Evaluation
4. Verify an Evaluation Proof

## Step 1: Setup

The first step is an one-time trusted setup and once it has done once, the following steps can be done repeatedly
1. Let $G_1$ and $G_2$ be pairing-friendly elliptic curve groups, determined by curve BLS12-381
2. Let $g_1$ be a generator of $G_1$ and $g_2$ be a generator of $G_2$
3. Let $l$ be the maximum degree of the polymonials we want to commit to ($l < p$)
4. Pick a random field element as secret parameter $\tau \in F_p$(usually done by MPC, to simplify, we just randomly choose one here)
5. Compute pp(public parameters, including proving key $pk$, and verifying key $vk$)$$pk = (g_1, g_1^\tau, g_1^{\tau^2},...,g_1^{\tau^l}), vk = g_2^\tau$$ and release it publicly
6. Discard secret parameter $\tau$ once the setup ceremony is done so that nobody can figure out its value

In [53]:
from py_ecc.bls12_381 import G1, G2, Z1, multiply, add
from py_ecc.fields import bls12_381_FQ as FQ
import random

# 1. Let G1,G2 be pairing-friendly elliptic curve groups, which are determined by the curve

# 2. Let g be a generator of G
g1 = G1
g2 = G2

# 3. Let l be the maximum degree of the polymonials, which is 16
l = 16

# 4. Pick a random field element as secret parameter t
p = 0x1a0111ea397fe69a4b1ba7b6434bacd764774b84f38512bf6730d2a0f6b0f6241eabfffeb153ffffb9feffffffffaaab
t = FQ(random.randint(0, p))
print(f'secret parameter 𝜏 is {t}')

# 5. Compute pp(public parameters)
def compute_public_parameters(g1, g2, t, l):

    pk = []
    accumulated = 1
    t_scalar = t
    for i in range(l + 1):
        # calculate g1, g1^t, g1^{t^2}...
        pk.append(multiply(g1, int(accumulated)))
        # calculate the exponential t, t^2, t^3, ...
        accumulated = accumulated * t_scalar
    vk = multiply(g2, int(t))
    return pk, vk

pk, vk = compute_public_parameters(g1, g2, t, l)
print(f'proving key pk: {pk}')
print(f'verifying key vk: {vk}')

secret parameter 𝜏 is 3107226914116588173371102363914303610635837725115045341757244990573439898766335531471856781517458490000508007703381
proving key pk: [(3685416753713387016781088315183077757961620795782546409894578378688607592378376318836054947676345821548104185464507, 1339506544944476473020471379941921221584933875938349620426543736416511423956333506472724655353366534992391756441569), (3999296978194622056425640900238434258005590916495432175355068086350356162778350846241539109637248209938138012770409, 818547637737456227721150021885336730115783426774728887102396497834862202704048902974963541067559345454575183216612), (1575121229301834256096529849484357719939965643190402210215805631466134510638553448151093383710941546885041746638926, 2299451848134577487004573966562307932115474667560885316973554602665870931612204347702396341060971511130972151584310), (3850675411069582138746663233947421051528595809980631139491731011738564127021503553629390154064456662934691625917083, 35465531826026346617

## Step 2: Commit to Polynomials
In reality, we arithmetize circuits and use Plonkish to get polynomials in this step
1. Given a polynomial $\Phi(x) = \sum_{i=0}^l \phi_i x^i$
2. Compute and output commitment $c = g^{\Phi(\tau)}$
   - Wait! $\tau$ has already been discard right? How can committer compute $\tau$?
   - Although he cannot compute $\Phi(\tau)$ directly, he can use public parameters to help with it:
$$\prod_{i=0}^l(g^{\tau^i})^{\phi_i} = g^{\sum_{i=0}^l \phi_i \tau^i} = g^{\Phi(\tau)}$$

In [54]:
# assume we have a polynomial Φ(𝑥)=4x^3+5x^2+3x+2 or some points on the polynomials
# we can use FFT or IFFT to do the transformation between the two forms
poly_coefs = [2, 3, 5, 4]

In [78]:
# compute commitment of the polynomial
def poly_commitment(pk, g1, poly_coefs):

  com = Z1
  for i in range(len(poly_coefs)):
    com = add(multiply(pk[i], poly_coefs[i]), com)
  return com

com = poly_commitment(pk, g1, poly_coefs)
print(f'commitment of polynomial is {com}')

commitment of polynomial is (3076324292172923587060794086648098461163009893624556251948034874412142147294205031698386090193294066776714796014900, 3445379784338642258058330491946048425717788101831823547311533020186335952722032006721300278808307849597020621281936)


## Step 3: Prove an Evaluation
In this period, the Verifier will ask Prover to 'OPEN' the commitment $c$ to a random specific point $a \in F_p$, in other word, Prover have to evaluation $\Phi(x)$ and commit the result in the form of opening triplet $OT = (a, b, \pi)$
1. Given an evaluation $\Phi(a) = b$
2. Compute and output proof of the evaluation $\pi = g^{q(\tau)}$, where $q(x) := \frac{\Phi(x)-b}{x-a}$
    - $q(x)$ is quotient polynomial: if $\Phi(a) = b$, that means $a$ is a root of $\Phi(x) - b$
    - so $\Phi(x) - b$ can be expressed as $\Phi(x) - b = q(x)(x-a)$, $q(x)$ is a polynomial
    - on the other hand, $q(x)$ exists if and only if $\Phi(a) = b$, so the existence of this quotient polynomial therefore serves as a proof of the evaluation

In [79]:
# Verifier first choose the random point a
def evaluation_poly(a, poly_coefs, q):
  b = 0
  for i in range(len(poly_coefs)):
    b += (a**i)*(poly_coefs[i])
  return b % q

a = random.randint(0, q)
b = evaluation_poly(a, poly_coefs, q)
print(f'random element we chosen is {a}, the evaluation b is {b}')

random element we chosen is 17, the evaluation b is 2


In [ ]:
# compute q(x)
def quotient_poly(a, b, poly_coefs):


q_poly = ((poly-b)/(x-a)).numerator()
print(f'quotient polynomial is {q_poly}')

In [ ]:
# compute proof of the evaluation pi
pi = poly_commitment(pk, g1,  q_poly_coefs)
print(f'proof of the evaluation pi is {pi}')

# Plonk Implementation(Python Version)

## Problem Definition

We take **Square-Fibonacci** as an example to demonstrate the process of proof generation

Defination of Square-Fibonacci Problem:
- Let $f_0 = 1, f_1 = 1$
- For $i \ge 2$, define $f_i:=(f_{i-2})^2+(f_{i-1})^2 \ mod \ q$
    - $q$ is a large prime integer, used to bound the size of each element, so that it can be represented by some predetermined number of bits.

Let $n$ be some very large integer. For convenience, we assume $n$ is a power of 2

Let $k$ be the $n^{th}$ Square-Fibonacci number

**Our goal**: generate an efficiently-verifiable proof $\pi$ showing that indeed $k$ is the $n^{th}$ Square-Fibonacci number(i.e. $f_n = k$)

## Phases of Proof Generation
The Plonk-based proof generation consists of 3 steps:
1. Filling in the trace table
2. Committing to the trace table
3. Proving the trace table's correctness

In [24]:
# some basic statement
# we assume q = p here, that is polynomial is defined over p
q = p

# assume we hope to prove '8th Square-Fibonacci number is k'
n = 8
k = FQ(317754178345286893212434)

# according to defination, f(0) = 1, f(1) = 1
f_0 = FQ(1)
f_1 = FQ(1)

## Step 1: Filling in the Trace Table
The trace table is a 2-dimensional matrix where 'witness' or 'trace' is written down, that is (n rows * 5 cols)
- 5 columns:
    - $A, B, C$: represent witness data / private input, each row lists 3 sequential Sequare-Fibonacci numbers
        - e.g. the $i^{th}$ row $(f_i, f_{i+1}, f_{i+2})$ is a witness for $(i+2)^{th}$ Sequare-Fibonacci number
    - $S$: represents selector column, indicating a certain mathmatical relation should hold over the element of the row.
        - $1$ represents the first 3 elements of the row $(a, b, c)$ must satisfy $c = a^2 + b^2 \ mod \ q$
        - $0$ represents the condition does not need to be satisfied.
    - $P$: represents public inputs, inputs to the circuit that are public known.
        - e.g. the first two values of the sequence $f_0, f_1$ and $k$ as the value to be proved
- n rows: left a blank row, so that the height of the table becomes $n$, an even power of 2
    - $1^{st}$ row: $f_0, f_1, f_2, 1, f_0$
    - $2^{nd}$ row: $f_1, f_2, f_3, 1, f_1$
    - $3^{rd}$ row: $f_2, f_3, f_4, 1, k$
    - ...
    - $(n-2)^{th}$ row: $f_{n-2}, f_{n-1}, f_n, 1, "" $
    - $(n-1)^{th}$ row: $"", "", "", 0, ""$

Next step is to fill in the trace table: either copy or compute over $F_q$

In [36]:
# generate witness/fill in the trace table
def witness_generation(f_0, f_1, k, n):

    trace_table = []

    # init col A, B, C, S
    f_a = f_0
    f_b = f_1
    f_c = f_b
    S = FQ(1)

    for i in range(n-1):
        f_a = f_b
        f_b = f_c
        f_c = f_a**2 + f_b**2
        trace_table.append([f_a, f_b, f_c, S, -1])

    # add a blank row to get n row
    S = 0
    trace_table.append([-1, -1, -1, S, -1])

    # add public parameters
    trace_table[0][4] = f_0
    trace_table[1][4] = f_1
    trace_table[2][4] = k

    return trace_table

trace_table = witness_generation(f_0, f_1, k, n)

print(f"trace table is:")
for A, B, C, S, P in trace_table:
    print("{:25} {:25} {:25} {:25} {:25}".format(int(A), int(B), int(C), int(S), int(P)))

trace table is:
                        1                         1                         2                         1                         1
                        1                         2                         5                         1                         1
                        2                         5                        29                         1  317754178345286893212434
                        5                        29                       866                         1                        -1
                       29                       866                    750797                         1                        -1
                      866                    750797              563696885165                         1                        -1
                   750797              563696885165  317754178345286893212434                         1                        -1
                       -1                        -1                       

## Step 2: Commit to the Trace Table

### interpret the trace table columns as polynomials

Each column can be considered as a length-$n$ vector of finite field elements $\rightarrow$ this vector can be regarded as the evaluation form of a polynomial $A(x)$ with degree $(n-1)$: the $i^{th}$ element of $A$ corresponds to the evaluation $A(\omega^i)$, where $\omega \in F_q$ is **$n^{th}$ root of unity** and has order $n$

In [37]:
# find a subgroup whose order = n
# def find_nth_root_of_unity(F_q, n):

#     while True:
#         w = F_q.random_element()
#         if w.multiplicative_order() == n:
#             break
#         else:
#             print(f'w = {w}, whose order is {w.multiplicative_order()}, not statified')
#     return w

# w = find_nth_root_of_unity(F_q, n)
# print(w)

# find a group: {1,2,4,7,8,11,13,14}
w_list = [1,2,4,7,8,11,13,14]

# interpreting the trace table columns as polynomials, and represent the polynomials as evaluation form
evaluation_form = []
for row in range(len(trace_table)):
    evaluation_form_row = []
    for col in range(len(trace_table[0])):
        val = trace_table[row][col]
        evaluation_form_row.append((w_list[row], val))
    evaluation_form.append(evaluation_form_row)
print(evaluation_form)

[[(1, 1), (1, 1), (1, 2), (1, 1), (1, 1)], [(2, 1), (2, 2), (2, 5), (2, 1), (2, 1)], [(4, 2), (4, 5), (4, 29), (4, 1), (4, 317754178345286893212434)], [(7, 5), (7, 29), (7, 866), (7, 1), (7, -1)], [(8, 29), (8, 866), (8, 750797), (8, 1), (8, -1)], [(11, 866), (11, 750797), (11, 563696885165), (11, 1), (11, -1)], [(13, 750797), (13, 563696885165), (13, 317754178345286893212434), (13, 1), (13, -1)], [(14, -1), (14, -1), (14, -1), (14, 0), (14, -1)]]
